In [12]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy.special as special
import time as time
import datetime as dt

from bs4 import BeautifulSoup
import requests
import re

In [13]:
# get bestseller links
url = 'http://www.yes24.com/24/Category/BestSeller' 
result = requests.get(url)
raw_html = BeautifulSoup(result.text, 'html.parser')

a_html = raw_html.find('ol')
list_bestseller_data = a_html.find_all('p', class_='copy')

bestseller_urls = [] 
for i in list_bestseller_data:
    urls_in_html = i.find('a')
    bestseller_urls.append('http://www.yes24.com/' + urls_in_html.get('href'))

In [14]:
# set frame for data
data = []

# get data for each best seller, create 2D array
for url in bestseller_urls: 
    result = requests.get(url) # get html
    raw_html = BeautifulSoup(result.text, 'html.parser')

    title = raw_html.find(name='h2', class_='gd_name').string

    author_wrapper = raw_html.find(name='span', class_='gd_auth')
    author = author_wrapper.find('a').string

    publisher_wrapper = raw_html.find(name='span', class_='gd_pub')
    publisher = publisher_wrapper.find('a').string

    published_date_str = raw_html.find(name='span', class_='gd_date').string
    published_date_num = published_date_str.translate({ord(i): None for i in '월년일'})

    origin_wrapper = raw_html.find(name='span', class_='gd_orgin')
    if origin_wrapper != None:
        origin = origin_wrapper.find('a').string

    review_score = raw_html.find(name='em', class_='yes_b').string
    review_number = raw_html.find(name='em', class_='txC_blue').string

    fixed_price = raw_html.find_all(name='em', class_='yes_m')[0]
    fixed_price = fixed_price.string
    selling_price = raw_html.find_all(name='em', class_='yes_m')[1]
    selling_price = selling_price.string

    ISBN13_parent_html = raw_html.find(name='th', class_='txt', string='ISBN13').parent
    ISBN13 = ISBN13_parent_html.find(name='td', class_='txt lastCol').string

    ISBN10_parent_html = raw_html.find(name='th', class_='txt', string='ISBN10').parent
    ISBN10 = ISBN10_parent_html.find(name='td', class_='txt lastCol').string

    yes24_sales_text = raw_html.find(name ='span', class_='gd_sellNum')
    yes24_sales_text = yes24_sales_text.text
    yes24_sales_index = ''
    for i in yes24_sales_text:
        if i.isnumeric() == True:
            yes24_sales_index = yes24_sales_index + i

    genre_list = raw_html.find(name='em', class_='bl_dot bgYUI').parent
    final_genre_name = genre_list.find_all('a')[-1].string

    total_data = [title, author, publisher, published_date_num, origin, review_score, review_number, fixed_price, selling_price, ISBN13, ISBN10, yes24_sales_index, final_genre_name]
    data.append(total_data)

In [15]:
# create dataframe with values
columns=['Title', 'Author', 'Publisher', 'Published_date', 'Origin', 'Review_score', 'Review_number', 'Fixed_price', 'Selling_price', 'ISBN13', 'ISBN10', 'Yes24_sales_index', 'Final_genre_name']
number_bestsellers = len(bestseller_urls)
rank = np.arange(1,number_bestsellers+1)
bestseller_df = pd.DataFrame(data=data, index=rank, columns=columns)
bestseller_df['Published_date'] = pd.to_datetime(bestseller_df['Published_date'], format='%Y %m %d')

In [16]:
bestseller_df.to_csv('bestseller.csv', encoding='CP949')